<a href="https://colab.research.google.com/github/Gregcub/NBA-Injury-Finder/blob/main/parse_injury.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pprint
%load_ext autoreload
%autoreload 2

Pretty printing has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/Shareddrives/injuryfindercolab
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g52OwTODw2VUaOWLrUbfdWLCbi6HEgVLjuWmaH6n-XcQ2meVDB7kPg
Mounted at /content/drive
/content/drive/Shareddrives/injuryfindercolab
data/  game.py  parse_injury.ipynb  __pycache__/


In [ ]:
from game import game

In [ ]:
class injury:
    def __init__(self, line):
        self.description = line['Injury']
        self.playername = line['Player Involved']
        self.date = datetime.strptime(line['Date of Injury/Last Game Played'], '%m/%d/%Y')
        self.gameid = None
        self.playerid = players[line['Player Involved']]
        self.teamid = teams[line['Team of Player Injured']]
        self.opposingteamid = teams[line['Opposing Team of Player Injured']]
        
        if (self.date,  self.teamid) in games.keys():
            self.gameid = games[(self.date, self.teamid)][0]
        elif (self.date,  self.opposingteamid) in games.keys():
            self.gameid = games[(self.date, self.opposingteamid)][0]
        else:
            pass
            # raise Exception('game not found')

    def as_dict(self):
        return {
            'description': self.description,
            'playername': self.playername,
            'date': self.date,
            'gameid': self.gameid,
            'playerid': self.playerid,
            'teamid': self.teamid,
            'opposingteamid': self.opposingteamid
        }
        

# Inverted Dictionary to ID injuries

In [ ]:
#{(DATE, HOMETEAMID): (GAMEID, VISITORTEAMID)}
games = {(datetime.strptime(row[1], '%Y/%m/%d'), row[2]) : (row[0], row[3]) for _, row in pd.read_csv('data/game.csv', dtype={'gameid':str, 'date':str, 'home':str, 'visitor':str}).iterrows()}
#{FULL NAME: (PLAYERID, TEAMID)}
players = {f'{row[3]} {row[2]}' : (row[0], row[1]) for _, row in pd.read_csv('data/player.csv', dtype={'playerid':str, 'teamid':str, 'jersey':str}).iterrows()}
#{TEAM NAME: TEAMID}
teams = {row[1] : row[0] for _, row in pd.read_csv('data/team.csv', dtype={'teamid':str}).iterrows()}

In [ ]:
errs = []
injuries = []
import csv
f = open('data/injuries.csv', 'r')
reader = csv.DictReader(f)
for line in reader:
    try:
        i = injury(line)
        if not i.gameid: continue
        injuries.append(i)
    except Exception as e:
        line['error'] = e
        errs.append(line)
f.close()
errsdf = pd.DataFrame(errs)

In [15]:
pd.DataFrame([i.as_dict() for i in injuries])

,description,playername,date,gameid,playerid,teamid,opposingteamid
0,Right Foot Fracture,Jodie Meeks,2015-10-28,0021500007,"(201975, 1610612765)",1610612765,1610612762
1,Achilles Injury,John Henson,2015-10-28,0021500012,"(203089, 1610612749)",1610612749,1610612752
2,Left Hip Injury,Kirk Hinrich,2015-11-27,0021500231,"(2550, 1610612741)",1610612741,1610612754
3,Right MCL Sprain,Robert Covington,2015-11-02,0021500046,"(203496, 1610612755)",1610612755,1610612739
4,Right Knee Injury,J.R. Smith,2015-11-02,0021500046,"(2747, 1610612739)",1610612739,1610612755
...,...,...,...,...,...,...,...
110,Right Knee Injury,Dirk Nowitzki,2016-01-18,0021500623,"(1717, 1610612742)",1610612742,1610612738
111,Left Ankle Sprain,Dwight Howard,2016-01-20,0021500635,"(2730, 1610612745)",1610612745,1610612765
112,Left Hip Injury,Brandon Knight,2016-01-19,0021500628,"(202688, 1610612756)",1610612756,1610612754
113,sprained left ankle,Mirza Teletovic,2016-01-19,0021500628,"(203141, 1610612756)",1610612756,1610612754


In [ ]:
with pd.option_context('display.max_rows', None):
    display(errsdf)

,Injury,Date of Injury/Last Game Played,Player Involved,Team of Player Injured,Opposing Team of Player Injured,When Player Returned from Injury,Second Injury,Date of Second Injury/Last Game Played,When Returned to Play,Third Injury,Date of Third Injury/Last Game Played,Fourth Injury,Date of Fourth Injury/Last Game Played,Fifth Injury,Date of Fifth Injury/Last Game Played,,error
0,Ankle Sprain,10/28/2014,Devin Booker,Phoenix Suns,Portland Trailblazers,11/4/2015,,,,,,,,,,,'Portland Trailblazers'
1,Strained Achilles,10/31/2015,Demarcus Cousins,Sacramento Kings,Los Angeles Clippers,11/9/2015,,,,,,,,,,,'Demarcus Cousins'
2,Sprained Left Ankle,11/2/2015,Michael Carter-Williams,Milwauke Bucks,Brooklyn Nets,11/14/2015,,,,,,,,,,,'Milwauke Bucks'
3,Left Ankle Fracture,12/26/2015,Alex Burks,Utah Jazz,Los Angeles Clippers,Did not Return,,,,,,,,,,,'Alex Burks'
4,Right Heel Injury,11/6/2015,Demarre Carroll,Toronto Raptors,Orlando magic,11/13/2015,Bruised Right Knee,12/5/2015,,Right Knee Injury,1/3/2016,,,,,,'Demarre Carroll'
5,Ankle Injury,11/16/2015,Chris Kaman,Portland Trailblazers,San Antonio Spurs,12/26/2015,,,,,,,,,,,'Portland Trailblazers'
6,Right Ankle Sprain,11/25/2015,J.J. Barea,Dallas Mavericks,San Antonio Spurs,12/9/2015,,,,,,,,,,,'J.J. Barea'
7,Right Ankle Injury,11/27/2015,Greivis Vasquez,Milwaukee Bucks,Orlando magic,3/30/3016,,,,,,,,,,,'Orlando magic'
8,Left Calf Injury,11/9/2015,C.J. Watson,Orlando Magic,Indiana Pacers,2/19/2016,,,,,,,,,,,'C.J. Watson'
9,Achilles Injury,11/13/2015,Wesley Matthews Jr.,Dallas Mavericks,Los Angeles lakers,11/16/2015,,,,,,,,,,,'Wesley Matthews Jr.'


In [ ]:
drive.flush_and_unmount()